# **01. Data Collection**

## Objectives

* Fetch data from Kaggle and prepare it for further processes. 

## Inputs

* Kaggle JSON file - the authentication token. 

## Outputs

* Generate Dataset: inputs/datasets/cherry_leaf_dataset

## Additional Comments

* Data will be downloaded, cleaned then split into train, validation and test datasets.



---

# Import Packages

In [4]:

%pip install -r /workspaces/Mildew_Detection_in_Cherry_Leaves/requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy
import os

# Change working directory

As the notebooks are located in a subfolder, the working directory needs to be changed from the current folder to the parent folder.


In [1]:
current_dir = os.getcwd()
current_dir

'/workspaces/Mildew_Detection_in_Cherry_Leaves/jupyter_notebooks'

The parent folder is made the new directory.

In [3]:
os.chdir('/workspaces/Mildew_Detection_in_Cherry_Leaves')
print("You set a new current directory")

You set a new current directory


The new current directory is confirmed

In [2]:
current_dir = os.getcwd()
current_dir

'/workspaces/Mildew_Detection_in_Cherry_Leaves/jupyter_notebooks'

# Install and Fetch Data from Kaggle

Install Kaggle

In [12]:
%pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 864.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 8.2 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.13-py3-none-any.whl size=77717 sha256=bbc0c82175d40878590b3ef9422e0f85407f6bee98e1861ce2452ef9a6b4e8cf
  Stored in directory: /home/codeany/.cache/pip/wheels/e6/8e/67/e07554a720a493dc6b39b30488590ba92ed45448ad0134d253
Successfully built kaggle
Note: you may need to restart the kernel to use updated packages.


Change the Kaggle configuration directory to the current working directory and set permissions for the kaggle authentication JSON.

In [13]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Set the Kaggle Dataset and download it to the inputs folder.

In [15]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry_leaf_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 89%|█████████████████████████████████▊    | 49.0M/55.0M [00:01<00:00, 36.8MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:01<00:00, 42.8MB/s]


Unzip the downloaded file, and delete the zip file.

In [16]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

# Data Preparation

## Data Cleaning

Check for non-image files and remove.

In [17]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        # print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location)  # remove non image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file", len(j))
        print(f"Folder: {folder} - has non-image file", len(i))

In [19]:
remove_non_image_file(my_data_dir='inputs/cherry_leaf_dataset/cherry-leaves')

Folder: healthy - has image file 2104
Folder: healthy - has non-image file 0
Folder: powdery_mildew - has image file 2104
Folder: powdery_mildew - has non-image file 0


## Split Data into Train, validation & Test Datasets

In [20]:
import os
import shutil
import random
import joblib


def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):

    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
        return

    # gets classes labels
    labels = os.listdir(my_data_dir)
    if 'test' in labels:
        pass
    else:
        # create train, test folders with classes labels sub-folder
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        for label in labels:

            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # move a given file to the train set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/' + label + '/' + file_name)

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # move a given file to the validation set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/' + file_name)

                else:
                    # move given file to test set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/' + file_name)

                count += 1

            os.rmdir(my_data_dir + '/' + label)


* The training set is divided into a 0.70 ratio of the data.
* The validation set is divided into a 0.10 ratio of the data.
* The test set is divided into a 0.20 ratio of the data.

In [21]:
split_train_validation_test_images(my_data_dir=f"inputs/cherry_leaf_dataset/cherry-leaves",
                                   train_set_ratio=0.7,
                                   validation_set_ratio=0.1,
                                   test_set_ratio=0.2
                                   )

# Conclusions and Next Steps

* The data has been downloaded, cleaned and sorted into relevant datasets.

* The next step is data visualization.